In [2]:
from my_utils.stardisting import load_published_he_model

In [3]:
model = load_published_he_model(
    folder_to_write_new_model_folder=r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\Segmentation Models\StarDist",
    name_for_new_model='2D_versatile_he')

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.
Using default values: prob_thresh=0.5, nms_thresh=0.4.

Ignore that, thresholds are: Thresholds(prob=0.6924782541382084, nms=0.3)
